In [32]:
from braindecode.datasets import MOABBDataset
from braindecode.preprocessing import preprocess, Preprocessor


dataset = MOABBDataset(dataset_name="BNCI2014_001", subject_ids=[1])


In [51]:
import numpy as np

from braindecode.preprocessing import (
    exponential_moving_standardize,
    preprocess,
    Preprocessor,
)

low_cut_hz = 4.0  # low cut frequency for filtering
high_cut_hz = 38.0  # high cut frequency for filtering
# Parameters for exponential moving standardization
factor_new = 1e-3
init_block_size = 1000

preprocessors = [
    Preprocessor("pick_types", eeg=True, meg=False, stim=False),  # Keep EEG sensors
    Preprocessor(
        lambda data, factor: np.multiply(data, factor),  # Convert from V to uV
        factor=1e6,
    ),
    Preprocessor("filter", l_freq=low_cut_hz, h_freq=high_cut_hz),  # Bandpass filter
    Preprocessor(
        exponential_moving_standardize,  # Exponential moving standardization
        factor_new=factor_new,
        init_block_size=init_block_size,
    ),
]

# Preprocess the data
preprocess(dataset, preprocessors, n_jobs=-1)

C:\Users\Joel\AppData\Roaming\Python\Python311\site-packages\braindecode\preprocessing\preprocess.py:55: UserWarning: Preprocessing choices with lambda functions cannot be saved.
  warn('Preprocessing choices with lambda functions cannot be saved.')


In [52]:
preprocessors = [
    Preprocessor('pick_types', eeg=True, meg=False, stim=True),
    Preprocessor('resample', sfreq=100)
]
# Preprocess the data
preprocess(dataset, preprocessors, n_jobs=-1)

In [55]:
from braindecode.preprocessing import create_windows_from_events

trial_start_offset_seconds = -0.5
# Extract sampling frequency, check that they are same in all datasets
sfreq = dataset.datasets[0].raw.info["sfreq"]
assert all([ds.raw.info["sfreq"] == sfreq for ds in dataset.datasets])
# Calculate the window start offset in samples.
trial_start_offset_samples = int(trial_start_offset_seconds * sfreq)

# Create windows using braindecode function for this. It needs parameters to
# define how windows should be used.
windows_dataset = create_windows_from_events(
    dataset,
    trial_start_offset_samples=trial_start_offset_samples,
    trial_stop_offset_samples=0,
    preload=True,
)

Used Annotations descriptions: ['feet', 'left_hand', 'right_hand', 'tongue']
Used Annotations descriptions: ['feet', 'left_hand', 'right_hand', 'tongue']
Used Annotations descriptions: ['feet', 'left_hand', 'right_hand', 'tongue']
Used Annotations descriptions: ['feet', 'left_hand', 'right_hand', 'tongue']
Used Annotations descriptions: ['feet', 'left_hand', 'right_hand', 'tongue']
Used Annotations descriptions: ['feet', 'left_hand', 'right_hand', 'tongue']
Used Annotations descriptions: ['feet', 'left_hand', 'right_hand', 'tongue']
Used Annotations descriptions: ['feet', 'left_hand', 'right_hand', 'tongue']
Used Annotations descriptions: ['feet', 'left_hand', 'right_hand', 'tongue']
Used Annotations descriptions: ['feet', 'left_hand', 'right_hand', 'tongue']
Used Annotations descriptions: ['feet', 'left_hand', 'right_hand', 'tongue']
Used Annotations descriptions: ['feet', 'left_hand', 'right_hand', 'tongue']


In [56]:
splitted = windows_dataset.split("session")
train_set = splitted['0train']  # Session train
test_set = splitted['1test']  # Session evaluation

In [58]:
import torch
from braindecode.util import set_random_seeds
from braindecode.models import ShallowFBCSPNet

cuda = torch.cuda.is_available()  # check if GPU is available, if True chooses to use it
device = "cuda" if cuda else "cpu"
if cuda:
    torch.backends.cudnn.benchmark = True
seed = 20200220
set_random_seeds(seed=seed, cuda=cuda)

n_classes = 4
classes = list(range(n_classes))
# Extract number of chans and time steps from dataset
n_channels = windows_dataset[0][0].shape[0]
input_window_samples = windows_dataset[0][0].shape[1]

model = ShallowFBCSPNet(
    n_channels,
    n_classes,
    n_times=input_window_samples,
    final_conv_length="auto",
)

# Display torchinfo table describing the model
print(model)

# Send model to GPU
if cuda:
    model.cuda()

Layer (type (var_name):depth-idx)        Input Shape               Output Shape              Param #                   Kernel Shape
ShallowFBCSPNet (ShallowFBCSPNet)        [1, 22, 450]              [1, 4]                    --                        --
├─Ensure4d (ensuredims): 1-1             [1, 22, 450]              [1, 22, 450, 1]           --                        --
├─Rearrange (dimshuffle): 1-2            [1, 22, 450, 1]           [1, 1, 450, 22]           --                        --
├─CombinedConv (conv_time_spat): 1-3     [1, 1, 450, 22]           [1, 40, 426, 1]           36,240                    --
├─BatchNorm2d (bnorm): 1-4               [1, 40, 426, 1]           [1, 40, 426, 1]           80                        --
├─Expression (conv_nonlin_exp): 1-5      [1, 40, 426, 1]           [1, 40, 426, 1]           --                        --
├─AvgPool2d (pool): 1-6                  [1, 40, 426, 1]           [1, 40, 24, 1]            --                        [75, 1]
├─Express

C:\Users\Joel\AppData\Roaming\Python\Python311\site-packages\braindecode\models\base.py:180: UserWarning: LogSoftmax final layer will be removed! Please adjust your loss function accordingly (e.g. CrossEntropyLoss)!
  warnings.warn("LogSoftmax final layer will be removed! " +


In [61]:
from skorch.callbacks import LRScheduler

from braindecode import EEGClassifier

lr = 0.0625 * 0.01
weight_decay = 0
batch_size = 64
n_epochs = 50

clf = EEGClassifier(
    model,
    criterion=torch.nn.NLLLoss,
    optimizer=torch.optim.AdamW,
    train_split=None,
    optimizer__lr=lr,
    optimizer__weight_decay=weight_decay,
    batch_size=batch_size,
    callbacks=[
        "accuracy",
        ("lr_scheduler", LRScheduler("CosineAnnealingLR", T_max=n_epochs - 1)),
    ],
    device=device,
    classes=classes,
    max_epochs=n_epochs,
)
# Model training for a specified number of epochs. `y` is None as it is already supplied
# in the dataset.
clf.fit(train_set, y=None)

# evaluated the model after training
y_test = test_set.get_metadata().target
test_acc = clf.score(test_set, y=y_test)
print(f"Test acc: {(test_acc * 100):.2f}%")

  epoch    train_accuracy    train_loss      lr     dur
-------  ----------------  ------------  ------  ------
      1            0.8785        0.7475  0.0006  0.6693
      2            0.8715        0.6937  0.0006  0.6391
      3            0.8819        0.6790  0.0006  0.4959
      4            0.8993        0.6036  0.0006  0.4515
      5            0.9167        0.6354  0.0006  0.4696
      6            0.9306        0.5919  0.0006  0.5745
      7            0.9340        0.5205  0.0006  0.6547
      8            0.9444        0.5116  0.0006  0.6027
      9            0.9340        0.4739  0.0006  0.6232
     10            0.9167        0.4681  0.0006  0.6467
     11            0.9167        0.4313  0.0006  0.6612
     12            0.9514        0.4178  0.0006  0.7192
     13            0.9653        0.4125  0.0005  0.7691
     14            0.9757        0.3888  0.0005  1.0391
     15            0.9757        0.3712  0.0005  0.7461
     16            0.9861        0.3835  0.0005 

In [73]:
all_preds = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14,1,2,3,4,5,6,7,8,9,10,11]
predicted = clf.predict(test_set)
print(predicted)
print(test_set)

[2 2 2 0 2 3 1 3 2 3 2 3 2 1 3 0 3 3 0 1 0 2 0 0 3 3 2 0 3 3 2 0 1 2 2 1 2
 3 1 1 0 3 3 0 3 0 3 0 0 3 1 2 3 1 0 0 1 0 1 1 2 3 2 0 1 3 3 3 2 2 3 3 1 0
 3 2 3 0 2 1 2 3 0 3 2 3 1 3 2 0 2 3 3 2 1 3 2 2 3 2 1 1 1 3 2 3 1 3 3 1 3
 2 3 3 2 0 2 0 2 3 2 3 1 2 2 3 3 0 0 0 3 3 3 1 3 2 0 3 1 2 2 0 3 1 2 3 2 3
 2 3 2 1 3 2 0 3 0 1 1 3 2 3 1 3 0 2 0 3 1 2 3 0 3 2 0 3 1 2 1 0 2 2 1 1 3
 3 0 0 0 1 2 0 2 2 2 0 0 2 0 3 2 0 1 1 2 0 3 2 3 3 0 0 3 2 2 3 2 2 1 1 0 0
 1 0 2 2 2 0 1 0 2 0 3 0 0 1 0 0 2 0 0 1 3 2 3 2 2 0 3 2 3 2 2 0 2 3 0 2 2
 1 2 3 3 0 3 2 2 0 2 2 3 2 0 3 2 2 1 3 3 3 2 3 0 3 0 0 2 3]
